In [12]:
using Ipopt
using ForwardDiff
using ReverseDiff

In [313]:
function ip_solve(x0::AbstractArray{T},f_obj,h_eq,g_ineq,num_h,num_g,x_min,x_max) where T
    num_x = length(x0)
    x_L = x_min
    x_U = x_max
    g_L = vcat(0. * ones(num_h), -1e19 * ones(num_g))
    g_U = vcat(0. * ones(num_h), 0. * ones(num_g))

    eval_f = x̃ -> f_obj(x̃)
    eval_grad_f = (x̃,grad_f) -> grad_f[:] = ForwardDiff.gradient(eval_f,x̃)[:]

    eval_g = (x̃,g̃) -> g̃[:] = vcat(h_eq(x̃),g_ineq(x̃))[:]
    function eval_jac_g(x, mode, rows, cols, values)
        if mode == :Structure
            for i = 1:num_h+num_g
                for j = 1:num_x
                    rows[(i-1)*num_x+j] = i
                    cols[(i-1)*num_x+j] = j
                end
            end
        else
            J = vcat(ForwardDiff.jacobian(h_eq,x),ForwardDiff.jacobian(g_ineq,x))
            values[:] = J'[:]
        end
    end

    prob = createProblem(num_x,x_L,x_U,
                         num_h+num_g,g_L,g_U,
                         num_x*(num_g+num_h),0,
                         eval_f,eval_g,
                         eval_grad_f,eval_jac_g)
    prob.x[:] = x0[:]

    addOption(prob, "hessian_approximation", "limited-memory")
    addOption(prob, "print_level", 0)
    status = solveProblem(prob)
    println(Ipopt.ApplicationReturnStatus[status])

    prob.x
end

ip_solve (generic function with 3 methods)

In [346]:
function softmax(x,y;k=1.)
    log.(exp.(k*x) + exp.(k*y))/k
end

function L(x,λ,h,c)
    f(x) - dot(λ,h(x)) + .5*c*dot(h(x),h(x))
end

function auglag_solve(x0,λ0,μ0,f0,h0,g0,x_min,x_max)
    num_h0 = length(λ0)
    num_g0 = length(μ0)
    num_x0 = length(x0)
    
    x = vcat(x0,zeros(num_g0))
    num_x = length(x)
    λ = vcat(λ0,μ0)
    num_h = length(λ)
    
    h = x̃ -> vcat(h0(x̃[1:num_x0]),g0(x̃[1:num_x0]) + x̃[num_x0+1:num_x0+num_g0].^2)
    f = x̃ -> f0(x̃[1:num_x0])
    
    num_steps = 10
    c = 1.
    k = 10.
    
    for i = 1:num_steps
        hx = h(x)
        ∇h = ForwardDiff.jacobian(h,x)
        ∇f = ForwardDiff.gradient(f,x)
        Hf = ForwardDiff.hessian(f,x)
        gL = ∇f + ∇h'*λ + c*∇h'*hx
        HL = Hf + c*∇h'*∇h
    
        A = vcat(hcat(HL,∇h'),hcat(∇h,zeros(num_h,num_h)))
        δ = (A+eye(num_x+num_h)*1e-12)\(-vcat(gL,hx))
        x += δ[1:num_x]
        λ += δ[num_x+1:num_x+num_h]
        
        x[1:num_x0] = -softmax.(-x[1:num_x0],-x_max,k=k)
        x[1:num_x0] = softmax.(x[1:num_x0],x_min,k=k)
    end
    
    x[1:num_x0]
end

auglag_solve (generic function with 3 methods)

In [347]:
function f(x)
    return x[1] * x[4] * (x[1] + x[2] + x[3]) + x[3]
end

function h(x)
    h = []
    h = vcat(h,[40. - (x[1]^2 + x[2]^2 + x[3]^2 + x[4]^2)])
    h = vcat(h, x[4] - x[2])
    h = vcat(h, x[1] * x[3])
    h = vcat(h, x[1] + x[2] + x[4] - 1.)
    h
end

function g(x)
    g = []
    g = vcat(g, x[4] * x[1])
    g
end

x0 = [-.5, 1., 1., 1.]
λ0 = ones(length(h(x0)))
μ0 = ones(length(g(x0)))

x_min = -1.*ones(length(x0))
x_max = 10.*ones(length(x0))

x_sol = auglag_solve(x0,λ0,μ0,f,h,g,x_min,x_max)

num_h = length(λ0)
num_g = length(μ0)
x_sol_ip = ip_solve(x0,f,h,g,num_h,num_g,x_min,x_max)

display(x_sol)
display(x_sol_ip)

display("Aug Lag")
display(f(x_sol))
display(h(x_sol))
display(g(x_sol))
display("IP")
display(f(x_sol_ip))
display(h(x_sol_ip))
display(g(x_sol_ip))

4-element Array{Float64,1}:
 6.03556e-5
 0.499971  
 6.28492   
 0.499859  

4-element Array{Float64,1}:
 4.45523e-12
 0.5        
 6.2849     
 0.5        

"Aug Lag"

6.2851208122245525

4-element Array{Any,1}:
 -1.48283e-6 
 -0.00011191 
  0.00037933 
 -0.000108756

...

"IP"

6.284902545005155

4-element Array{Any,1}:
 -6.50857e-12
  5.82501e-12
  2.80007e-11
 -1.13354e-11

1-element Array{Any,1}:
 2.22762e-12

Solve_Succeeded
